# Feuille de travaux pratiques. Premières applications du calcul scientifique

In [ ]:
# chargement des bibliothèques
import numpy as np

## Exercice 1 (calcul d'une valeur approchée de $\pi$ par la méthode de Monte-Carlo)

Pour obtenir une valeur approchée du nombre $\pi$ par la [méthode de Monte-Carlo](https://fr.wikipedia.org/wiki/M%C3%A9thode_de_Monte-Carlo), on tire « au hasard », dans un carré de côté de longueur égale à $2$, des points de coordonnées $(x,y)$ et l'on vérifie s'ils appartiennent ou non au disque de rayon égal à $1$ ayant même centre que le carré. Les points étant tirés dans l'ensemble du carré avec la même loi de probabilité, le rapport entre le nombre de points tirés dans le disque et le nombre de points tirés au total tend, lorsque le nombre de tirages tend vers l'infini, vers le rapport des surfaces du disque et du carré, soit $\frac{\pi}{4}$, en vertu de la [loi des grands nombres](https://fr.wikipedia.org/wiki/Loi_des_grands_nombres).

**1.** Au moyen de la commande `random.rand` de NumPy, qui génère une suite de nombres réels jouant le rôle d'une réalisation d'une suite de variables aléatoires continues, indépendantes et identiquement distribuées selon la [loi uniforme](https://fr.wikipedia.org/wiki/Loi_uniforme_continue) sur l'intervalle $[0,1]$, écrire une fonction prenant comme argument le nombre de tirages à réaliser et renvoyant la valeur approchée de $\pi$ obtenue par la méthode de Monte-Carlo décrite ci-dessus (pour simplifier, on pourra se restreindre au quart de carré contenu dans l'orthant positif de $\mathbb{R}^2$).

La méthode consiste à tirer $N$ couples $(x,y)$ dans $[0,1]^2$, à vérifier s'ils appartiennent au disque unité, à diviser le nombre de fois où c'est le cas par le nombre total de tirages et enfin à multiplier ce résultat par $4$.

In [ ]:
def Pibymontecarlo(N):
# calcul d'une estimation du nombre pi par la méthode de Monte-Carlo à partir de N tirages
    count=0
    for i in range(N):
        x,y=np.random.rand(2)
        if (pow(x,2)+pow(y,2)<1):
            count=count+1
    return 4*count/N

Testons la fonction.

In [ ]:
# test pour 100 tirages
approximation=Pibymontecarlo(100)
print('Approximation obtenue pour pour 100 tirages :',approximation)
print('Erreur relative :',abs(approximation-np.pi)/np.pi)

**2.** Donner un ordre du nombre de tirages nécessaires pour obtenir plus de deux décimales exactes de $\pi$. Que dire de l'efficacité de cette méthode ?

Tirer un point aléatoirement et vérifier s'il appartient au cercle est une [épreuve de Bernoulli](http://fr.wikipedia.org/wiki/%C3%89preuve_de_Bernoulli) de probabilité $\frac{\pi}{4}$. Par le [théorème de la limite centrale](http://fr.wikipedia.org/wiki/Th%C3%A9or%C3%A8me_central_limite), l'écart-type d'une moyenne de $N$ expériences aléatoires identiques décroît comme $\frac{1}{\sqrt{N}}$. Il faut donc de l'ordre de $N=10^4$ tirages pour obtenir deux décimales exactes,

In [ ]:
for i in range(10):
      print('Erreur relative :',abs(Pibymontecarlo(10000)-np.pi)/np.pi)

et de l'ordre de $N=10^6$ tirages pour en obtenir trois.

In [ ]:
for i in range(10):
      print('Erreur relative :',abs(Pibymontecarlo(1000000)-np.pi)/np.pi)

## Exercice 2 (procédé d'orthonormalisation de Gram-Schmidt)

On rappelle que, partant d'une famille $\mathcal{B}=\left\{u_1,\dots,u_m\right\}$ de vecteurs linéairement indépendants de $\mathbb{R}^n$, avec $m$ et $n$ des entiers
tels que $2\leq m\leq n$, le [procédé d'orthonormalisation de Gram-Schmidt](https://en.wikipedia.org/wiki/Gram%E2%80%93Schmidt_process) permet de construire
une famille $\mathcal{B}'=\left\{q_1,\dots,q_m\right\}$ de vecteurs orthonormaux donnés par
$$
q_1=\frac{u_1}{\|u_1\|_2},
$$
$$
\widetilde{q}_{k+1}=u_{k+1}-\sum_{i=1}^k\left<u_{k+1},q_i\right>q_i,\ q_{k+1}=\frac{\widetilde{q}_{k+1}}{\|\widetilde{q}_{k+1}\|_2},\ k=1,\dots,m-1,
$$
où $\|\cdot\|_2$ et $\left<\cdot\,,\cdot\right>$ désignent respectivement la norme euclidienne et le produit scalaire euclidien sur $\mathbb{R}^n$.

**1.** Écrire une fonction nommée `gramschmidt`, prenant comme paramètre d'entrée un tableau ayant pour colonnes les $m$ vecteurs de la famille $\mathcal{B}$ et renvoyant un tableau ayant pour colonnes les $m$ vecteurs de la famille $\mathcal{B}'$, obtenue en appliquant à $\mathcal{B}$ le procédé d'orthonormalisation de Gram-Schmidt. Penser à inclure un mécanisme vérifiant que la famille $\mathcal{B}$ fournie lors de l'appel de la fonction est libre.

In [ ]:
def gramschmidt(A):
# calcul d'une famille orthonormale de vecteurs à partir d'une famille libre par le procédé d'orthonormalisation de Gram-Schmidt
    n,m=A.shape
    Q=np.zeros((n,m))
    for k in range(m):
        Q[:,k]=A[:,k]
        for i in range(k):
            Q[:,k]=Q[:,k]-np.dot(Q[:,i],A[:,k])*Q[:,i]
        norme=np.linalg.norm(Q[:,k],2)
        if norme<1e-14:
            raise ValueError('La famille n\'est apparemment pas libre.')
        else:
            Q[:,k]=Q[:,k]/norme
    return Q

**2.** On pose $\varepsilon=10^{-8}$. Tester la fonction `gramschmidt` avec la famille
$$
\mathcal{B}=\left\{\begin{pmatrix}1\\\varepsilon\\0\\0\end{pmatrix},\begin{pmatrix}1\\0\\\varepsilon\\0\end{pmatrix},\begin{pmatrix}1\\0\\0\\\varepsilon\end{pmatrix}\right\},
$$
puis vérifier que les vecteurs obtenus sont bien orthogonaux deux à deux. Que constate-t-on ?

In [ ]:
epsilon=1e-8
B=np.array([[1,1,1],[epsilon,0,0],[0,epsilon,0],[0,0,epsilon]])
Q=gramschmidt(B)

On calcule la [matrice de Gram](http://fr.wikipedia.org/wiki/D%C3%A9terminant_de_Gram#Matrice_de_Gram) de la famille $\mathcal{B}'$ de vecteurs obtenue, qui doit être égale à la matrice identité si les vecteurs sont orthonormaux.

In [ ]:
print(np.dot(Q.T,Q))

On constate que les deux derniers vecteurs ne sont pas orthogonaux. Ce problème de stabilité numérique est dû au fait que les calculs sont effectués en arithmétique en précision finie et que les vecteurs de la famille $\mathcal{B}$ sont «presque» liés, le réel $\varepsilon$ étant de l'ordre de la racine de l'[epsilon machine](http://en.wikipedia.org/wiki/Machine_epsilon).

**3.** Pour pallier le défaut d'orthogonalité des vecteurs de la famille $\mathcal{B}'$ (qui sont dûs à l'accumulation des erreurs d'arrondi, les calculs étant faits en arithmétique à virgule flottante), il faut utiliser une version (numériquement) plus stable du procédé. Celle-ci consiste à opérer de la manière suivante :
$$
q_1=\frac{u_1}{\|u_1\|_2},
$$
$$
q^{(0)}_{k+1}=u_{k+1},\ q^{(i)}_{k+1}=q^{(i-1)}_{k+1}-\left<q^{(i-1)}_{k+1},q_i\right>q_i,\ i=1,\dots,k,\ q_{k+1}=\frac{q^{(k)}_{k+1}}{\|q^{(k)}_{k+1}\|_2},\ k=1,\dots,m-1.
$$
Mettre en &oelig;uvre, en modifiant la fonction déjà existante, cette variante pour obtenir une nouvelle fonction qu'on nommera `modgramschmidt` (pour procédé de Gram-Schmidt *modifié*).
Effectuer ensuite l'orthonormalisation de la famille $\mathcal{B}$ donnée dans la question précédente.

In [ ]:
def modgramschmidt(A):
# calcul d'une famille orthonormale de vecteurs à partir d'une famille libre par le procédé d'orthonormalisation de Gram-Schmidt modifié
    n,m=A.shape
    Q=np.zeros((n,m))
    for k in range(m):
        norme=np.linalg.norm(A[:,k],2)
        if norme<1e-14:
            raise ValueError('La famille n\'est apparemment pas libre.')
        else:
            Q[:,k]=A[:,k]/norme
        for i in range(k+1,m):
            A[:,i]=A[:,i]-np.dot(Q[:,k],A[:,i])*Q[:,k]
    return Q

Reprenons l'orthonormalisation de la famille $\mathcal{B}$.

In [ ]:
Q=modgramschmidt(B)
print(np.dot(Q.T,Q))

Cette version du procédé est plus stable numériquement. Même avec un réel $\varepsilon$ plus petit, on n'observe plus de problème.

In [ ]:
epsilon=1e-12
B=np.array([[1,1,1],[epsilon,0,0],[0,epsilon,0],[0,0,epsilon]])
Q=modgramschmidt(B)
print(np.dot(Q.T,Q))

## Exercice 3 (applications linéaires entre espaces de polynômes)

Soit $n$ un entier naturel. On note $\mathbb{R}_n[X]$ l'espace vectoriel des polynômes à coefficients réels de degré inférieur ou égal à $n$.

**1.** Pour $P$ et $Q$ deux éléments de $\mathbb{R}_n[X]$, on note $R(P,Q)$ le reste de la division euclidienne de $P$ par $Q$. On considére l'application linéaire :
$$
\begin{array}{rcccc}
\mathcal{R}&:&\mathbb{R}_n[X]&\rightarrow&\mathbb{R}_n[X]\\
&&P&\mapsto&R(P,X^2).
\end{array}
$$
**(a)** Écrire une fonction prenant comme paramètre un entier $n$ et renvoyant la matrice $M_\mathcal{R}$ de l'application $\mathcal{R}$ dans la base canonique de $\mathbb{R}_n[X]$.

Le reste de la [division euclidienne](http://fr.wikipedia.org/wiki/Division_euclidienne) de $X^n$ par $X^2$ est $1$ si $n=0$, $X$ si $n=1$, et $0$ sinon. On obtient donc la fonction suivante.

In [ ]:
def remaindermatrix(n):
# assemblage de la matrice (dans la base canonique de R_n[X]) de l'application linéaire donnant le reste de la division euclidienne d'un polynôme de degré inférieur ou égal à n par X^2
    if n==0:
        M=np.array([1])
    elif n==1:
        M=np.eye(2)
    else:
        M=np.diag(np.concatenate((np.ones(2),np.zeros(n-1))))
    return M

Effectuons un test élémentaire. Le reste de la division euclidienne de $X^3+2\,X^2+3\,X+4$ est $3\,X+4$.

In [ ]:
P=np.array([4,3,2,1])
print(np.dot(remaindermatrix(3),P))

**(b)** À l'aide de cette fonction, calculer le reste de la division par $X^2$ pour les polynômes suivants :
$$
7\,X^8+411\,X^7-231\,X^5+31\,X^4+451\,X^3-231\,X-42,
$$
$$
X^7+\frac{5}{21}\,X^5+0,432\,X^4-22\,X^3+51\,X^2-\frac{1}{39}\,X+4,431.
$$

In [ ]:
P=np.array([-42,-231,0,451,31,-231,0,411,7])
print(np.dot(remaindermatrix(P.size-1),P))
P=np.array([4.431,1/39,51,-22,0.432,5/21,0,1])
print(np.dot(remaindermatrix(P.size-1),P))

On trouve respectivement $−231\,X−42$ et $0,02564103\,X+4,431$.

**(c)** En utilisant la fonction `linalg.null_space` de SciPy, déterminer le noyau de l'application $\mathcal{R}$ pour $n=6$, $7$ et $8$. Que constate-t-on ?

In [ ]:
from scipy.linalg import null_space
for n in range(6,9):
    print(null_space(remaindermatrix(n)))

Le noyau de l'application linéaire $\mathcal{R}$ est formé des polynômes ne possédant pas de terme de degré inférieur ou égal à un.

**2.** On considère à présent l'application linéaire dérivée :
$$
\begin{array}{rcccc}
\mathcal{D}&:&\mathbb{R}_n[X]&\rightarrow&\mathbb{R}_n[X]\\
&&P&\mapsto&P',
\end{array}
$$
associant à tout polynôme $P$ de $\mathbb{R}_n[X]$ ayant pour coefficients $a_k$, $k=0,\dots,n$, le polynôme $P'$ ayant pour fonction polynomiale $P'(x)=\sum_{k=1}^nk\,a_k\,x^{k-1}$. Reprendre la question précédente avec $\mathcal{D}$ en place de $\mathcal{R}$.

Pour tout entier naturel $n$, la dérivée de $X^n$ est $n\,X^{n−1}$. On a par conséquent la fonction suivante.

In [ ]:
def derivativematrix(n):
# assemblage de la matrice (dans la base canonique de R_n[X]) de l'application linéaire donnant la dérivée d'un polynôme de degré inférieur ou égal à n
    if n==0:
        M=np.zeros(1)
    else:
        M=np.diag(np.arange(1,n+1),1)
    return M

Calculons le polynôme dérivé de $P(x)=7\,X^8+411\,X^7-231\,X^5+31\,X^4+451\,X^3-231\,X-42$.

In [ ]:
P=np.array([-42,-231,0,451,31,-231,0,411,7])
print(np.dot(derivativematrix(P.size-1),P))

On trouve $P'(X)=56\,X^7+2877\,X^6−1155\,X^4+124\,X^3+1353\,X^2−231$.

In [ ]:
for n in range(6,9):
    print(null_space(derivativematrix(n)))

On trouve que le noyau de l'application $\mathcal{D}$ est constitué des constantes.

**3.** On considère enfin les deux applications composées $\mathcal{D}\circ\mathcal{R}$ et $\mathcal{R}\circ\mathcal{D}$. Que font-elles ? Sont-elles identiques ? Quel est leur lien avec les produits de matrices $M_\mathcal{D}M_\mathcal{R}$ et $M_\mathcal{R}M_\mathcal{D}$ ?

L'application composée $\mathcal{D}\circ\mathcal{R}$ associe à un polynôme la dérivée du reste de sa division euclidienne par $X^2$, alors que l'application composée $\mathcal{R}\circ\mathcal{D}$ associe à un polynôme la dérivée du reste de sa division euclidienne par $X^2$, ce qui n'est généralement pas la même chose. Par exemple, on a $\mathcal{D}\circ\mathcal{R}(X^2)=0$, mais $\mathcal{R}\circ\mathcal{D}(X^2)=2\,X$.

In [ ]:
M_D=derivativematrix(2)
M_R=remaindermatrix(2)
P=np.array([0,0,1])
print(np.dot(np.dot(M_D,M_R),P))
print(np.dot(np.dot(M_R,M_D),P))

Ceci se traduit par le fait que les matrices $M_\mathcal{D}$ et $M_\mathcal{R}$ ne commutent pas. On mesure ce défaut de commutation en calculant le commutateur $[M_\mathcal{D},M_\mathcal{R}]=M_\mathcal{D}M_\mathcal{R}−M_\mathcal{R}M_\mathcal{D}$.

In [ ]:
print(np.dot(M_D,M_R)-np.dot(M_R,M_D))